# CNN with CIFAR10

CIFAR 10은 컬러 데이터로, MNIST와는 다르게 channel이 3개가 됩니다.

CIFAR 10부터는 GPU 변경을 해주시는 것을 추천드립니다.

## 과제 설명

아래 물음표로 되어 있는 곳을 채워서 제출하시고, 다양한 hyperparameter 변경, 혹은 다양한 기법을 통해서 성능을 높이는 것이 최종 목표입니다.
물론, 수업 시간에 진행한 VGG만큼의 성능은 얻을 수 없지만, 최대한 다양한 방법을 통해 성능을 높여서 제출해주세요.

4월 19일 수요일 자정까지 CNN, 그리고 추가적으로 배울 GAN 모델을 CIFAR-10에 대해서 구현해보는 것이 목표입니다.
과제: 공지드린 바와 같이 이번주 과제는 ML_Day4_1, ML_Day4_2 파일의 빈칸을 완성하여 ds.slcf@gmail.com으로 제출해주시면 됩니다.
과제 혹은 강의 내용 관련 문의사항이 있으실 경우 자유롭게 문의 주시기 바랍니다!

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# Define the normalization statistics
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# Define the data transforms for the training set
train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])
# Define the data transforms for the validation/test set
valid_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])
# Load the CIFAR10 training set
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_tfms)
# Load the CIFAR10 validation/test set
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=valid_tfms)

from torch.utils.data import random_split
batch_size = 128
# Define the size of the validation set as a percentage of the total dataset
val_size = 0.2
# Calculate the size of the validation set based on the validation size and the total size of the dataset
num_train = len(train_data)
num_val = int(val_size * num_train)
# Split the training set into a training set and a validation set
train_data, val_data = random_split(train_data, [num_train - num_val, num_val])
# Create data loaders for the training, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)
# Define the class labels
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


## 이번에는 직접 모델을 만들어 보도록 하겠습니다.

MNIST 실습파일과는 다르게 정답이 정해지지 않았으며, 오류가 나지 않도록 원하시는 대로 숫자를 넣어 코드를 돌리고 제출해주세요!

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # 32x32x3 -> 32x32x16
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(16, 32, 3, padding=1),  # 32x32x16 -> 32x32x32
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(32, 64, 3, padding=1),  # 32x32x32 -> 32x32x64
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(64, 128, 3, padding=1),  # 32x32x64 -> 32x32x128
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(128, 256, 3, padding=1),  # 32x32x128 -> 32x32x256
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(2, 2),  # 32x32x256 -> 16x16x256
            nn.Conv2d(256, 512, 3, padding=1),  # 16x16x256 -> 16x16x512
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(512, 1024, 3, padding=1),  # 16x16x512 -> 16x16x1024
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(2, 2),  # 16x16x1024 -> 8x8x1024
            nn.Conv2d(1024, 2048, 3, padding=1),  # 8x8x1024 -> 8x8x2048
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(2, 2)  # 8x8x2048 -> 4x4x2048
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(4*4*2048, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        out = self.conv_layer(x)
        out = out.view(-1, 4*4*2048)
        out = self.fc_layer(out)

        return out


In [8]:
model = CNN().cuda()

In [9]:
loss = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

## 14.3 Train Model

In [10]:
## val loss 까지 계산 -> overfitting 확인용

num_epochs = 40
batch_size = 200

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):

    # Train
    model.train()
    train_loss = 0.0
    total_batch = len(train_data) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        train_loss += cost.item()

        if (i+1) % 200 == 0:
            print('Epoch [%d/%d], Train Iter [%d/%d], Train Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, train_loss/200))
            train_loss = 0.0

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_images, batch_labels in val_loader:
            X = batch_images.cuda()
            Y = batch_labels.cuda()

            pre = model(X)
            cost = loss(pre, Y)

            val_loss += cost.item()

    val_loss /= len(val_loader)
    print('Epoch [%d/%d], Validation Loss: %.4f' %(epoch+1, num_epochs, val_loss))


Epoch [1/40], Train Iter [200/200], Train Loss: 2.0901
Epoch [1/40], Validation Loss: 1.8202
Epoch [2/40], Train Iter [200/200], Train Loss: 1.6737
Epoch [2/40], Validation Loss: 1.5831
Epoch [3/40], Train Iter [200/200], Train Loss: 1.4770
Epoch [3/40], Validation Loss: 1.4015
Epoch [4/40], Train Iter [200/200], Train Loss: 1.3402
Epoch [4/40], Validation Loss: 1.2837
Epoch [5/40], Train Iter [200/200], Train Loss: 1.2246
Epoch [5/40], Validation Loss: 1.2245
Epoch [6/40], Train Iter [200/200], Train Loss: 1.1416
Epoch [6/40], Validation Loss: 1.1100
Epoch [7/40], Train Iter [200/200], Train Loss: 1.0614
Epoch [7/40], Validation Loss: 1.0334
Epoch [8/40], Train Iter [200/200], Train Loss: 0.9992
Epoch [8/40], Validation Loss: 0.9639
Epoch [9/40], Train Iter [200/200], Train Loss: 0.9449
Epoch [9/40], Validation Loss: 0.9067
Epoch [10/40], Train Iter [200/200], Train Loss: 0.8916
Epoch [10/40], Validation Loss: 0.8393
Epoch [11/40], Train Iter [200/200], Train Loss: 0.8388
Epoch [11/40

## 14.4 Test Model

In [12]:
correct = 0
total = 0
model.eval() # 검증 모드 
with torch.no_grad(): # 역전파 x

    for images, labels in test_loader:

        images = images.cuda()
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 85.720000 %
